In [1]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Parameters
image_size = 224
batch_size = 48  # Increase batch size
num_classes = 2  # maize and turmeric
learning_rate = 0.001 
num_epochs = 50 

In [3]:
# Data transformation
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x + 1.95 * torch.randn_like(x)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
# Dataset directories
output_folder = './split-dataset-2'
train_dataset = datasets.ImageFolder(os.path.join(output_folder, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(output_folder, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(output_folder, 'test'), transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Load pre-trained DenseNet-121 model
model = models.densenet121(pretrained=True)

# Modify the classifier to match the number of output classes (maize and turmeric)
model.classifier = nn.Linear(model.classifier.in_features, num_classes)

# Move model to the configured device
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

C:\Users\cse10\miniconda3\envs\data-science-env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\cse10\miniconda3\envs\data-science-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Training loop
val_accuracies = []
train_losses = []
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch + 1}, Training Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")
    val_accuracies.append(accuracy)

print("Training complete.")

Epoch 1/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:50<00:00,  2.41it/s]


Epoch 1, Training Loss: 0.2115
Validation Accuracy: 92.49%


Epoch 2/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:46<00:00,  2.47it/s]


Epoch 2, Training Loss: 0.1557
Validation Accuracy: 94.17%


Epoch 3/50: 100%|████████████████████████████████████████████████████████████████████| 411/411 [02:47<00:00,  2.46it/s]


...

Epoch 48, Training Loss: 0.0594
Validation Accuracy: 97.13%


Epoch 49/50: 100%|███████████████████████████████████████████████████████████████████| 411/411 [02:47<00:00,  2.46it/s]


Epoch 49, Training Loss: 0.0579
Validation Accuracy: 97.01%


Epoch 50/50: 100%|███████████████████████████████████████████████████████████████████| 411/411 [02:47<00:00,  2.45it/s]


Epoch 50, Training Loss: 0.0588
Validation Accuracy: 96.54%
Training complete.


In [8]:
import pandas as pd
val_df = pd.DataFrame({
    "Epoch": list(range(1, num_epochs + 1)),
    "Validation Accuracy": val_accuracies,
    "Training Loss": train_losses
})

# Append to a CSV file (ensure this file name is unique for each model)
val_df.to_csv("model-2_val_accuracy.csv", mode='a', index=False, header=not pd.io.common.file_exists("model-2_val_accuracy.csv"))

In [9]:
# Evaluation on the test set
model.eval()
true_labels = []
predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        true_labels.extend(labels.cpu().numpy())
        predictions.extend(predicted.cpu().numpy())

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='weighted')
recall = recall_score(true_labels, predictions, average='weighted')
f1 = f1_score(true_labels, predictions, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Test Accuracy: 0.9673
Precision: 0.9669
Recall: 0.9673
F1 Score: 0.9663


In [10]:
# In each model notebook, at the end of your metrics calculation, do this:
import pandas as pd

# Example metrics for Model 1
densenet_metrics = {
    "Model": "DenseNet-121",
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1,
}

metrics_df = pd.DataFrame([densenet_metrics])

# Append to CSV file
metrics_df.to_csv("model_metrics-2.csv", mode='a', index=False, header=not pd.io.common.file_exists("model_metrics-2.csv"))
